## Data Splits

In [1]:
!cd /app/project
!rm -rf /app/project/data/train
!rm -rf /app/project/data/val
!rm -rf /app/project/data/test
!python3 create_splits.py --source /app/project/data/waymo/training_and_validation/ --destination /app/project/data/
!echo " Training count"
!ls data/train | wc -l
!echo " Val count"
!ls data/val | wc -l
!echo " Test count"
!ls data/test | wc -l
!echo " Training Files"
!ls data/train
!echo " Test Files"
!ls data/test
!echo " Validation Files"
!ls data/val

2022-03-12 02:17:30,159 INFO     Creating splits...
72 15 10
Creating symlink for train files
Creating symlink for validation files
Creating symlink for testing files
 Training count
72
 Val count
15
 Test count
10
 Training Files
segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord
segment-10023947602400723454_1120_000_1140_000_with_camera_labels.tfrecord
segment-10061305430875486848_1080_000_1100_000_with_camera_labels.tfrecord
segment-10072140764565668044_4060_000_4080_000_with_camera_labels.tfrecord
segment-10075870402459732738_1060_000_1080_000_with_camera_labels.tfrecord
segment-10094743350625019937_3420_000_3440_000_with_camera_labels.tfrecord
segment-10096619443888687526_2820_000_2840_000_with_camera_labels.tfrecord
segment-10107710434105775874_760_000_780_000_with_camera_labels.tfrecord
segment-10153695247769592104_787_000_807_000_with_camera_labels.tfrecord
segment-10206293520369375008_2796_800_2816_800_with_camera_labels.tfrecord
segment-102124064984970

## To Create the config file

In [4]:
!rm -rf /app/project/training/reference
!mkdir -p /app/project/training/reference
!python edit_config.py --train_dir /app/project/data/train/ --eval_dir /app/project/data/val/ --batch_size 4 \
--checkpoint ./training/pretrained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0 \
--label_map label_map.pbtxt
!cp pipeline_new.config /app/project/training/reference

## To Limit the GPU memory

In [1]:
'''
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
'''
pass

Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPUs


In [4]:
#!pip install gast==0.3.3
!nvidia-smi

Sat Mar 12 14:48:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 31%   56C    P2    89W / 160W |   5456MiB /  6144MiB |     97%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
# Training Process
#!python /app/project/experiments/model_main_tf2.py --model_dir=training/reference/ \
#--pipeline_config_path=training/reference/pipeline_new.config 
pass

In [18]:
!ps -aux

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0   6000  3664 pts/0    Ss+  02:10   0:00 bash
root          10  0.0  0.3 293808 109248 pts/0   Sl   02:10   0:43 /usr/bin/pyth
root          15  0.0  0.1 774080 57772 ?        Ssl  02:10   0:09 /usr/bin/pyth
root          44  0.0  0.1 773860 59104 ?        Rsl  02:14   0:04 /usr/bin/pyth
root         513  4.0  1.2 2693528 397896 ?      Ssl  02:40  28:36 /usr/bin/pyth
root         744  0.0  0.0   2624    72 pts/1    Ss+  02:40   0:00 /usr/bin/sh -
root         745  0.0  1.3 9132708 443320 pts/1  Sl+  02:40   0:19 /usr/local/bi
root         763  0.0  0.3 1363552 113312 pts/1  Sl+  02:40   0:12 /usr/local/li
root        2002  1.7 18.6 20528716 6083560 ?    Ssl  13:01   1:38 /usr/bin/pyth
root        5536  143  7.3 7123096 2400312 pts/0 Rl   14:32   0:38 python experi
root        5795  0.0  0.0   2624   548 pts/2    Ss+  14:33   0:00 /usr/bin/sh -
root        5796  0.0  0.0   7656  3152 pts

In [8]:
# Limit the GPU memory
'''
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
'''
pass

In [9]:
#!python experiments/model_main_tf2.py --model_dir=training/reference/ --pipeline_config_path=training/reference/pipeline_new.config --checkpoint_dir=training/reference/
pass

## Save the Model

In [5]:
!python exporter_main_v2.py \
  --pipeline_config_path=training/reference/pipeline_new.config \
  --trained_checkpoint_dir=training/reference/ \
  --output_directory=experiments/Experiment1 \
  --input_type=image_tensor

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0312 21:34:39.163445 140281237657408 deprecation.py:610] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:458: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0312 21:34:52.806506 140281237657408 save_impl.py:71] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7f9540156d00>, because it is not built.
W0312 21:35:04.454685 140281237657408 

## Create a Movie

In [7]:
!python inference_video.py --labelmap_path label_map.pbtxt \
--model_path experiments/Experiment0/saved_model \
--tf_record_path data/waymo/training_and_validation/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord \
--config_path experiments/Experiment0/pipeline.config \
--output_path experiments/Experiment0/animation.mp4

2022-03-12 21:40:45,310 INFO     Loading model from experiments/Experiment0/saved_model
2022-03-12 21:40:52,278 INFO     Loading config from experiments/Experiment0/pipeline.config
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.deterministic`.
Instructions for updating:
Use `tf.data.Dataset.map()
2022-03-12 21:40:53,747 INFO     Inference on data/waymo/training_and_validation/segment-10017090168044687777_6380_000_6400_000_with_camera_labels.tfrecord
2022-03-12 21:40:53,829 INFO     Step: 0
2022-03-12 21:40:57,044 INFO     Step: 10
